In [1]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
import gzip
import time
import shutil
import urllib
import requests



In [2]:
def download():
  
    r = requests.get('http://data.dot.state.mn.us/iris_xml/incident.xml.gz')
    with open('incident.xml', 'w') as handle:
        handle.write(gzip.decompress(r.content).decode('utf-8'))

In [6]:
def data_check(XMLfile):
    try:
        with open('crash_data.csv', 'r') as CD:
            parse(XMLfile)
    except FileNotFoundError:
            All_Crash_Data = pd.DataFrame(columns=['Name', 'Date', 'Direction', 'Road', 'Location', 'Event'])
            with open('crash_data.csv', 'w') as f:
                All_Crash_Data.to_csv(f, header=True)
            parse(XMLfile)


In [27]:
def parse(XMLfile):
    with open('crash_data.csv', 'r') as CD:
           All_Crash_Data = pd.read_csv(CD)

    dates = []
    incident_dirs = []
    roads = []
    locations = []
    names = []
    events = []
    
    parsedXML = ET.parse(XMLfile)
    root = parsedXML.getroot()
    for child in root:
        if child.attrib['name'] not in str(All_Crash_Data['Name']):
            try:
                dates.append(child.attrib['event_date'])
            except KeyError:
                dates.append("none")
            try:
                names.append(str(child.attrib['name']))
            except KeyError:
                name.append("none")
            try:
                incident_dirs.append(child.attrib['dir'])
            except KeyError:
                incident_dir.append("none")
            try:
                roads.append(child.attrib['road'])
            except KeyError:
                roads.append('None')
            try:
                locations.append(child.attrib['location'])
            except KeyError:
                locations.append("none")
            try: 
                event = child.attrib['event_type'].split("_", 1)
                events.append(event[1])
            except KeyError:
                events.append("none")
  


    DF = pd.DataFrame({"Name" : names,
                       "Date" : dates,
                       "Direction": incident_dirs,
                       "Road" : roads,
                       "Location" : locations,
                       "Event" : events})
    
    print(DF)


    with open('crash_data.csv', 'a') as f:
        DF.to_csv(f, header=False)


In [28]:
data_check("incident.xml")

                Name                          Date Direction     Road  \
0   2018092819553371  Fri Sep 28 19:55:33 CDT 2018        WB   T.H.62   
1       L004_9940513  Sat Sep 29 19:56:42 CDT 2018        NB    I-35E   
2       L004_9940574  Sat Sep 29 20:41:42 CDT 2018        SB    I-35E   
3       L004_9940567  Sat Sep 29 20:32:52 CDT 2018        WB  T.H.212   
4   2018092819531862  Fri Sep 28 19:53:18 CDT 2018        SB  U.S.169   
5   2018092819533003  Fri Sep 28 19:53:30 CDT 2018        NB  U.S.169   
6       L004_9940577  Sat Sep 29 20:43:22 CDT 2018        NB  U.S.169   
7       L004_9940544  Sat Sep 29 20:19:32 CDT 2018        NB    I-35E   
8       L004_9940545  Sat Sep 29 20:20:02 CDT 2018        WB     I-90   
9   2018092819315453  Fri Sep 28 19:31:54 CDT 2018        NB  T.H.100   
10  2018092820101610  Fri Sep 28 20:10:16 CDT 2018        SB    I-35W   
11  2018092820040732  Fri Sep 28 20:04:07 CDT 2018        SB    I-35E   
12  2018092820090159  Fri Sep 28 20:09:01 CDT 2018 